# Classification of 3s and 8s using Neural Network and Logistic Regression

In [12]:
#hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
#hide
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import confusion_matrix
from mpl_toolkits import mplot3d

In [ ]:
#hide
!pip install mnist

In [14]:
#hide
import mnist

In [15]:
#hide
train_images = mnist.train_images()
train_labels = mnist.train_labels()

EOFError: Compressed file ended before the end-of-stream marker was reached

In [11]:
#hide
train_images.shape, train_labels.shape

NameError: name 'train_images' is not defined

In [ ]:
#hide
test_images = mnist.test_images()
test_labels = mnist.test_labels()

In [ ]:
#hide
test_images.shape, test_labels.shape

In [ ]:
#hide
image_index = 4 
print(train_labels[image_index]) 
plt.imshow(train_images[image_index], cmap='Greys');

In [ ]:
#hide
train_filter = np.where((train_labels == 3 ) | (train_labels == 8))
test_filter = np.where((test_labels == 3) | (test_labels == 8))
X_train, y_train = train_images[train_filter], train_labels[train_filter]
X_test, y_test = test_images[test_filter], test_labels[test_filter]
y_train

In [ ]:
#hide
X_train = X_train/255.
X_test = X_test/255.

In [ ]:
#hide
y_train1 = 1*(y_train==3)
y_test1 = 1*(y_test==3)
y_train1 = y_train1.reshape(-1,1)
y_test1 = y_test1.reshape(-1,1)

In [ ]:
#hide
y_test1

In [ ]:
#hide
X_train.shape, X_test.shape

In [ ]:
#hide
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)
X_train.shape, X_test.shape

# Config class

In [ ]:
class Config:
    pass
config = Config()
config.lr = 0.001
config.num_epochs = 220
config.bs = 50

# Importing Functions from kudzu library

In [ ]:
from kudzu.data import Data, Sampler, Dataloader
from kudzu.layer import Affine, Sigmoid, Relu
from kudzu.loss import BCE
from kudzu.model import Model
from kudzu.optim import GD
from kudzu.train import Learner
from kudzu.callbacks import AccCallback, take_mean

In [ ]:
#hide
layers = [Affine('first',784,100), Relu('first_Relu'), Affine('second',100,100), Relu('second_Relu'), Affine('third',100,2),Affine('fourth',2,1), Sigmoid('sigmoid')]

# ClfCallback class

In [ ]:
acc = AccCallback
class CLfCallback(acc):
    def __init__(self,learner,bs):
        super().__init__(learner,bs)
        self.accuracies = []
        self.test_accuracies = []
    
    def epoch_end(self, train_prob, test_prob, predicted_test):
        for layer, name, fnval, grval in self.learner.model.params_and_grads():
            self.paramhist[layer.name+'_'+name].append(fnval)
            self.gradhist[layer.name+'_'+name].append(grval)
        eloss = take_mean(self.batch_losses[-self.bpe:], self.bpe, self.afrac)
        self.losses.append(eloss)
        self.accuracies.append(train_prob)
        self.test_accuracies.append(test_prob)
        self.predictions = np.array(predicted_test)
        if self.epoch % 10 ==0:
            print(f"Epoch {self.epoch} Loss {eloss}\n train accuracy: {train_prob} test accuracy: {test_prob}\n\n")
        

    

# Neural Network model

In [ ]:
model = Model(layers)
data = Data(X_train, y_train1)
sampler = Sampler(data, config.bs, shuffle = True)
dl = Dataloader(data, sampler)
loss = BCE()
opt = GD(config.lr)


In [ ]:
learner = Learner(loss, model, opt, config.num_epochs)
ccb = CLfCallback(learner, config.bs)
learner.set_callbacks([ccb])

# Loop training process begins

In [ ]:
#hide_input
learner.train_loop(dl, X_test, y_test1)

In [ ]:
#hide_input
print(f'Maximum train set accuracy is {max(ccb.accuracies)}\n')
print(f'Maximum test set accuracy is {max(ccb.test_accuracies)}')

In [ ]:
#hide_input
plt.plot(ccb.accuracies, 'b', label = 'train accuracies')
plt.plot(ccb.test_accuracies,'g', label = 'test accuracies')
plt.legend();
plt.title('Accuracies plot of Neural Network\n');

In [ ]:
#hide_input
plt.plot(ccb.losses)
plt.title('Losses plot of Neural Network');

In [ ]:
#hide
new_model = Model(layers[:-2])
inputs = new_model(X_test)
x1,x2 = np.meshgrid(inputs[:,0], inputs[:,1])
w1 = ccb.get_weights('fourth',0)[-1]
w2 = ccb.get_weights('fourth',1)[-1]
b = ccb.get_biases('fourth')[-1]

In [ ]:
#hide
print(x1.max(), x1.min())
print(x2.max(), x2.min())

In [ ]:
#hide
x1grid = np.linspace(-4,4,500)
x2grid = np.linspace(9,-6,500)
X1, X2 = np.meshgrid(x1grid, x2grid)
line = w1*X1 + w2*X2 +b
prob = layers[-1](line)

In [ ]:
#hide
y_test.ravel()
b = np.argwhere(y_test ==3)
a = np.argwhere(y_test==8)

In [ ]:
#hide_input
plt.figure(figsize=(14,10))
plt.scatter(inputs[:,0][b],inputs[:,1][b], marker = 'x', alpha = 0.2, label = '3')
plt.scatter(inputs[:,0][a],inputs[:,1][a], marker = 'o', alpha = 0.1, label = '8')
plt.legend(loc = 'upper left')
contours = plt.contour(X1, X2, prob,10)
plt.clabel(contours,inline = True, fontsize = 10);
plt.colorbar(contours);
plt.title('CLassification of Data\n');

# Confusion Matrix for Neural Network

In [ ]:
#hide_input
matrix = confusion_matrix(y_test1,ccb.predictions)
print(matrix)
print(f'Number of False positives: {matrix[0,1]}\n')
print(f'Number of False negatives: {matrix[1,0]}')

# Logistic Regression model

In [ ]:
layers_logistic = [Affine('logistic_aff',784,1),Sigmoid('sigmoid_aff')]
logistic_model = Model(layers_logistic)
learner_logistic = Learner(loss, logistic_model, opt, config.num_epochs)
ccb_logistic = CLfCallback(learner_logistic, config.bs)
learner_logistic.set_callbacks([ccb_logistic])

# Training process of Logistic Regression model begins

In [ ]:
#hide_input
learner_logistic.train_loop(dl, X_test, y_test1)

In [ ]:
#hide_input
print(f'Maximum train set accuracy is {max(ccb_logistic.accuracies)}\n')
print(f'Maximum test set accuracy is {max(ccb_logistic.test_accuracies)}')

In [ ]:
#hide_input
plt.plot(ccb_logistic.accuracies, 'b', label = 'train accuracies')
plt.plot(ccb_logistic.test_accuracies,'g', label = 'test accuracies')
plt.legend();
plt.title('Accuracies plot of Logistic Regression model\n');

In [ ]:
#hide_input
plt.plot(ccb_logistic.losses)
plt.title('Losses plot of Logistic Regression\n');

# Confusion Matrix for Logistic Regression

In [ ]:
#hide_input
matrix1 = confusion_matrix(y_test1, ccb_logistic.predictions)
print(matrix1)
print(f'Number of False positives: {matrix1[0,1]}\n')
print(f'Number of False negatives: {matrix1[1,0]}')

By these plots we can observe that when performing logistic regression instead of Neural network, our accuracy is less in logistic regression but overfitting doesn't seem to happen in logistic regression
Observing the confusion matrix for both these models we can say that number of false results are also more in case of Logistic Regression model.